In [1]:
import numpy as np
import pandas as pd


In [ ]:
substitutes_df = pd.read_csv("/content/drive/MyDrive/Substitute.csv")

In [ ]:
substitutes_df

,Ingredient,Amount,Substitute
0,Allspice,1 tsp.,1/2 tsp. cinnamon and 1/2 tsp. ground cloves
1,Apple pie spice,1 tsp.,"1/2 tsp. cinnamon, 1/4 tsp. nutmeg, and 1/8 ts..."
2,Arrowroot,1 1/2 tsp.,1 tbsp. flour
3,Arrowroot,1 1/2 tsp.,1 1/2 tsp. cornstarch
4,Baking powder,1 tsp.,1/3 tsp. baking soda and 1/2 tsp. cream of tartar
...,...,...,...
157,Tomato juice,1 cup,1/2 cup tomato sauce and 1/2 cup water
158,Worcestershire sauce,1 tsp.,1 tsp. bottled steak sauce
159,"Yogurt, plain",1 cup,1 cup buttermilk
160,"Yogurt, plain",1 cup,1 cup cottage cheese blended until smooth


In [ ]:
recipes = pd.read_csv("/content/drive/MyDrive/total1.csv")

In [ ]:
print(recipes)

       Unnamed: 0                                      Recipe Title  \
0               0                       Double Chocolate Layer Cake   
1               1                          Salted Water for Boiling   
2               2                              Spiced Pumpkin Bread   
3               3                              Simple Roast Chicken   
4               4  Pan-Seared Tuna with Ginger-Shiitake Cream Sauce   
...           ...                                               ...   
45783       45783                               Sausage roll wreath   
45784       45784                Roast chicken with dill & potatoes   
45785       45785                    Polish chocolate & walnut cake   
45786       45786                                 Easy apricot tart   
45787       45787                                     Keto brownies   

                                             Ingredients  \
0      ['Serves 12 to 14', 'For cake layers', '3 oz. ...   
1                          

In [ ]:
!pip install nltk

In [ ]:
# Preprocess and tokenize the ingredients
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english') + list(string.punctuation))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Combine title, ingredients, and steps into a single text field
# Preprocess and combine the data
from sklearn.feature_extraction.text import TfidfVectorizer
def preprocess_text(text):
    # Basic text cleaning and tokenization
    tokenized = word_tokenize(text.lower())
    return ' '.join([word for word in tokenized if word not in stop_words])
recipes['combined_text'] = recipes.apply(lambda x: preprocess_text(x['Recipe Title'] + ' ' + ' '.join(x['Ingredients']) + ' ' + ' '.join(x['Steps'])), axis=1)

# Apply TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(recipes['combined_text'])

tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
print(tfidf_df)

       039s  045cream  045dill  045style   10   12   15  1977  1989   20  ...  \
0       0.0       0.0      0.0       0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...   
1       0.0       0.0      0.0       0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...   
2       0.0       0.0      0.0       0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...   
3       0.0       0.0      0.0       0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...   
4       0.0       0.0      0.0       0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...   
...     ...       ...      ...       ...  ...  ...  ...   ...   ...  ...  ...   
45783   0.0       0.0      0.0       0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...   
45784   0.0       0.0      0.0       0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...   
45785   0.0       0.0      0.0       0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...   
45786   0.0       0.0      0.0       0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...   
45787   0.0       0.0      0.0       0.0  0.0  0.0  0.0   0.0   0.0  0.0  ...   

       zongzi  zoodles  zoo

In [ ]:
tfidf_matrix

<45788x6787 sparse matrix of type '<class 'numpy.float64'>'
	with 183041 stored elements in Compressed Sparse Row format>

In [ ]:
import pandas as pd


# Create a dictionary with ingredients as keys and a list of substitutes as values
substitute_dict = {}
for index, row in substitutes_df.iterrows():
    ingredient = row['Ingredient'].strip().lower()
    substitute = row['Substitute'].strip()
    if ingredient in substitute_dict:
        substitute_dict[ingredient].append(substitute)
    else:
        substitute_dict[ingredient] = [substitute]

# Function to find substitutes
def find_substitutes(ingredient):
    ingredient = ingredient.lower()
    return substitute_dict.get(ingredient, ["No substitute found"])

# Example usage
print(find_substitutes("Baking powder"))

print(find_substitutes("Allspice"))


['1/3 tsp. baking soda and 1/2 tsp. cream of tartar', '1/4 tsp. baking soda and 1/2 cup sour milk or buttermilk (Decrease liquid called for in the recipe by 1/2 cup.)', '1/4 tsp. baking soda and 1/2 tsp. vinegar or lemon juice used with sweet milk to make 1/2 cup (Decrease liquid called for in the recipe by 1/2 cup.)']
['1/2 tsp. cinnamon and 1/2 tsp. ground cloves']


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
pip install rank-bm25


In [5]:
import pandas as pd
from rank_bm25 import BM25Okapi
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Load your dataset
df = pd.read_csv("/content/drive/MyDrive/total1.csv")

# Combining title, ingredients, and steps into a single text for each recipe
df['combined_text'] = df['Recipe Title'] + " " + df['Ingredients'].astype(str) + " " + df['Steps'].astype(str)

# Tokenization
tokenized_data = [doc.split(" ") for doc in df['combined_text']]

# Creating the BM25 object
bm25 = BM25Okapi(tokenized_data)

def search_recipes(query):
    query = query.split(" ")
    scores = bm25.get_scores(query)
    top_indexes = np.argsort(scores)[::-1][:10]  # Get top 10 indexes
    return df.iloc[top_indexes]

# Example query
search_results = search_recipes("chocolate cake")
print(search_results[['Recipe Title', 'Ingredients', 'Steps']])

                                            Recipe Title  \
6064                         Triple-Chocolate Layer Cake   
8528                              Chocolate Biscuit Cake   
41245                                Halloween cake pops   
23653                  Candy-Filled Chocolate Easter Egg   
42985                        Christmas pudding cake pops   
8346              Double-Chocolate Tangerine Mousse Cake   
7283                         Chocolate-Mint Brownie Cake   
45736                               Czech chocolate cake   
3729   Peanut Butter Cake with Chocolate-Peanut Butte...   
27670                                     Guacamole Cake   

                                             Ingredients  \
6064   ['Serves 8 to 10', 'For cake', '1/2 cup cold b...   
8528   ['Makes 8 servings', '1/2 teaspoon butter, for...   
41245  ['100g/4oz madeira cake', '100g chocolate sand...   
23653  ['One 9-by-1-inch confetti cake round', '1 pou...   
42985  ['200g madeira cake', '140g-160g

In [4]:
import pandas as pd
from rank_bm25 import BM25Okapi
import numpy as np

# Load your dataset
# df = pd.read_csv('your_dataset.csv')

# Preprocessing: assuming 'Ingredients' is a list of ingredients in string format
df['cleaned_ingredients'] = df['Ingredients'].apply(lambda x: x.lower().split(','))

# Tokenize: split each ingredient list into words
tokenized_ingredients = [doc for doc in df['cleaned_ingredients']]

# Create the BM25 object
bm25 = BM25Okapi(tokenized_ingredients)

def search_recipes_by_ingredients(ingredients):
    query = ingredients.lower().split(',')
    scores = bm25.get_scores(query)
    top_indexes = np.argsort(scores)[::-1][:10]  # Adjust number of results as needed
    return df.iloc[top_indexes]

# Example query: search for recipes with specific ingredients
search_results = search_recipes_by_ingredients("flour, sugar, eggs")
print(search_results[['Recipe Title', 'Ingredients', 'Steps']])


                                            Recipe Title  \
45787                                      Keto brownies   
15247  Hannah's Peanut Butter and White Chocolate Can...   
15267                              Japanese Cold Noodles   
15266                   Braised Shrimp and Creamy Endive   
15265        Fresh Orange Slices with Honey and Cinnamon   
15264  Turkey and Sweet-Potato Croquettes with Cranbe...   
15263               Chocolate-Orange Easter-Egg Truffles   
15262                               Chocolate Cheesecake   
15261                              Gruyère Walnut Wafers   
15260                 Apple Strudel with Cranberry Sauce   

                                             Ingredients  \
45787  ['120g nut butter (we used almond butter)', '1...   
15247  ['Makes about 80 pieces', 'Nonstick vegetable ...   
15267  ['Makes 6 servings', 'For dipping sauce', '2 c...   
15266  ['Makes 2 servings', '1 lb Belgian endives (4 ...   
15265  ['Makes 4 servings', '4 oranges'

In [5]:
# for GPU use
!CT_CUBLAS=1 pip install ctransformers --no-binary ctransformers

In [6]:
model_id = "TheBloke/Llama-2-13B-chat-GGML"

In [7]:
from ctransformers import AutoModelForCausalLM

# check ctransformers doc for more configs
config = {'max_new_tokens': 10000, 'repetition_penalty': 1.1,
          'temperature': 0.1, 'stream': True}

llm = AutoModelForCausalLM.from_pretrained(
      model_id,
      model_type="llama",
      #lib='avx2', for cpu use
      gpu_layers=130, #110 for 7b, 130 for 13b
      **config
      )

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
import pandas as pd
from rank_bm25 import BM25Okapi
import numpy as np
from ctransformers import AutoModelForCausalLM

# Assume df is your dataset containing recipes
# df = pd.read_csv('your_dataset.csv')

# Preprocessing for BM25
df['cleaned_ingredients'] = df['Ingredients'].apply(lambda x: x.lower().split(','))
tokenized_ingredients = [doc for doc in df['cleaned_ingredients']]
bm25 = BM25Okapi(tokenized_ingredients)

def search_recipes_by_ingredients(ingredients):
    query = ingredients.lower().split(',')
    scores = bm25.get_scores(query)
    top_indexes = np.argsort(scores)[::-1][:10]  # Adjust number of results as needed
    top_recipes = df.iloc[top_indexes]
    return top_recipes

# Load and configure the Quantized Llama model
model_id = "TheBloke/Llama-2-13B-chat-GGML"
llm = AutoModelForCausalLM.from_pretrained(
    model_id,
    model_type="llama",
    gpu_layers=130,  # Adjust based on your model
    # Other configurations
)

def generate_recipe(ingredients):
    search_results = search_recipes_by_ingredients(ingredients)
    top_recipe_titles = search_results['Recipe Title'].tolist()

    # Prepare the prompt for the generator model
    prompt = f"Generate a recipe using the following ingredients: {ingredients}. Inspired by: {', '.join(top_recipe_titles)}"

    # Generate recipe with Llama model
    tokens = llm.tokenize(prompt)
    generated_text = ''.join([llm.detokenize(token) for token in llm.generate(tokens)])

    return generated_text

# Example usage
ingredients = "flour, sugar, eggs"
recipe = generate_recipe(ingredients)
print(recipe)


Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.


KeyboardInterrupt: ignored